In [ ]:
import os
import pickle

import numpy as np
import pandas as pd
import polars as pl

import optuna
from optuna.samplers import TPESampler
import torch

from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import trustworthiness
import pacmap

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

os.chdir("/home/denisalpino/dev/FinABYSS/")
from utils.vizualization import show_feature_importance, show_hpo
from utils.metrics import calc_noiseless_silhouette

In [ ]:
print(f"Доступно GPU: {torch.cuda.device_count()}")
device = 0 if torch.cuda.is_available() else -1
device

### Loading sample of embeddings (10000, 768)

In [5]:
# Загрузка sample-выборки эмбеддингов (10000, 768)
embeddings = np.load('notebooks/aspects/data/embeddings.npy')
embeddings.shape

(9973, 768)

### Глоабальная оптимизация гиперпараметров PCA+HDBSCAN

In [ ]:
def objective_global(trial):
    # Параметры UMAP
    umap_n_neighbors = trial.suggest_int("umap_n_neighbors", 5, 100)
    umap_min_dist = trial.suggest_float("umap_min_dist", 0.0, 0.5, step=0.01)
    umap_n_components = trial.suggest_int("umap_n_components", 10, 80)

    # Применяем UMAP прямо к исходным эмбеддингам
    umap_model = UMAP(
        n_neighbors=umap_n_neighbors,
        min_dist=umap_min_dist,
        n_components=umap_n_components,
        metric='cosine'
    )
    embedding_intermediate = umap_model.fit_transform(embeddings)

    # Параметры HDBSCAN
    hdbscan_min_cluster_size = trial.suggest_int("hdbscan_min_cluster_size", 5, 100)
    hdbscan_min_samples = trial.suggest_int("hdbscan_min_samples", 1, hdbscan_min_cluster_size)
    cluster_selection_epsilon = trial.suggest_uniform("cluster_selection_epsilon", 0.0, 1.0)

    clusterer = HDBSCAN(
        min_cluster_size=hdbscan_min_cluster_size,
        min_samples=hdbscan_min_samples,
        cluster_selection_method='eom',
        cluster_selection_epsilon=cluster_selection_epsilon,
    )
    labels = clusterer.fit_predict(embedding_intermediate)

    # Вычисляем silhouette score только для ненойзовых точек
    valid = labels != -1
    if np.sum(valid) < 10 or len(np.unique(labels[valid])) < 2:
        return -1.0
    score = silhouette_score(embedding_intermediate[valid], labels[valid])
    return score

print("Запускаем глобальную оптимизацию...")
study_global = optuna.create_study(direction="maximize")
study_global.optimize(objective_global, n_trials=200, show_progress_bar=True)

print("Глобальная оптимизация завершена")
print("Лучшие глобальные гиперпараметры:")
for key, value in study_global.best_params.items():
    print(f"  {key}: {value}")
print("Лучший глобальный silhouette score:", study_global.best_value)

Глобальная оптимизация завершена 
Лучшие глобальные гиперпараметры: 
* umap_n_neighbors: 48
* umap_min_dist: 0.05
* umap_n_components: 10
* hdbscan_min_cluster_size: 15
* hdbscan_min_samples: 11
* cluster_selection_epsilon: 0.002172166608873129

Лучший глобальный silhouette score: 0.7065192461013794


### Локальная (refined) оптимизация гиперпараметров

In [ ]:
global_best = study_global.best_params

def refined_range(val, delta, low_bound, high_bound):
    """Функция для вычисления новых диапазонов вокруг глобальных лучших значений"""
    return (max(low_bound, val - delta), min(high_bound, val + delta))

# Задаём уточнённые диапазоны:
n_neighbors_range = refined_range(global_best["umap_n_neighbors"], 20, 5, 100)
min_dist_range = refined_range(global_best["umap_min_dist"], 0.05, 0.0, 0.5)
n_components_range = refined_range(global_best["umap_n_components"], 10, 10, 80)
hdbscan_cluster_range = refined_range(global_best["hdbscan_min_cluster_size"], 10, 5, 100)

def objective_refined(trial):
    # Параметры UMAP
    umap_n_neighbors = trial.suggest_int("umap_n_neighbors", n_neighbors_range[0], n_neighbors_range[1])
    umap_min_dist = trial.suggest_float("umap_min_dist", min_dist_range[0], min_dist_range[1], step=0.01)
    umap_n_components = trial.suggest_int("umap_n_components", n_components_range[0], n_components_range[1])

    umap_model = UMAP(
        n_neighbors=umap_n_neighbors,
        min_dist=umap_min_dist,
        n_components=umap_n_components,
        metric="cosine",
        random_state=42
    )
    embedding_intermediate = umap_model.fit_transform(embeddings)

    # Параметры HDBSCAN
    hdbscan_min_cluster_size = trial.suggest_int("hdbscan_min_cluster_size", hdbscan_cluster_range[0], hdbscan_cluster_range[1])
    hdbscan_min_samples = trial.suggest_int("hdbscan_min_samples", 1, hdbscan_min_cluster_size)
    cluster_selection_epsilon = trial.suggest_uniform("cluster_selection_epsilon", 0.0, 0.2)

    clusterer = HDBSCAN(
        min_cluster_size=hdbscan_min_cluster_size,
        min_samples=hdbscan_min_samples,
        cluster_selection_method='eom',
        cluster_selection_epsilon=cluster_selection_epsilon,
    )
    labels = clusterer.fit_predict(embedding_intermediate)

    valid = labels != -1
    if np.sum(valid) < 10 or len(np.unique(labels[valid])) < 2:
        return -1.0
    score = silhouette_score(embedding_intermediate[valid], labels[valid])
    return score

print("Запускаем уточненную оптимизацию...")
study_refined = optuna.create_study(direction="maximize")
study_refined.optimize(objective_refined, n_trials=100, show_progress_bar=True)

print("Уточненная оптимизация завершена")
print("Лучшие уточненные гиперпараметры:")
for key, value in study_refined.best_params.items():
    print(f"  {key}: {value}")
print("Лучший refined silhouette score:", study_refined.best_value)

Уточненная оптимизация завершена 
Лучшие уточненные гиперпараметры:
* umap_n_neighbors: 48
* umap_min_dist: 0.05
* umap_n_components: 15
* hdbscan_min_cluster_size: 16
* hdbscan_min_samples: 15
* cluster_selection_epsilon: 0.022176326579653287

Лучший refined silhouette score: 0.7122868299484253

In [ ]:
df_trials, fig = show_hpo(
    study_global,
    "silhouette_score",
    title="HPO combinations for UMAP & HDBSCAN (Random Search)",
    lib="optuna"
)

In [ ]:
df_trials.to_csv("hpo.csv")

### Финальное обучение модели с лучшими (refined) параметрами

In [ ]:
##############################################
# 4. Финальное обучение модели с лучшими (refined) параметрами
##############################################
refined_best = study_refined.best_params

# Финальный UMAP на исходных эмбеддингах
umap_final = UMAP(
    n_neighbors=refined_best["umap_n_neighbors"],
    min_dist=refined_best["umap_min_dist"],
    n_components=refined_best["umap_n_components"],
    metric="cosine",
    random_state=42
)
embedding_intermediate_final = umap_final.fit_transform(embeddings)

# Финальная кластеризация HDBSCAN
hdbscan_final = HDBSCAN(
    min_cluster_size=refined_best["hdbscan_min_cluster_size"],
    min_samples=refined_best["hdbscan_min_samples"],
    cluster_selection_epsilon=refined_best["cluster_selection_epsilon"],
    cluster_selection_method='eom',
)
final_labels = hdbscan_final.fit_predict(embedding_intermediate_final)
n_clusters = len(np.unique(final_labels[final_labels != -1]))
n_noise = np.sum(final_labels == -1)
print(f"Финальная кластеризация: кластеров = {n_clusters}, noise = {n_noise} точек")

Финальная кластеризация: кластеров = 86, noise = 3841 точек

In [ ]:
# Фильтруем данные: рассматриваем только ненойзовые точки
mask_valid = final_labels != -1
if np.sum(mask_valid) < 2:
    print("Недостаточно валидных точек для вычисления метрик.")
else:
    # Берем промежуточное представление, использованное для финальной кластеризации
    X_valid = embedding_intermediate_final[mask_valid]
    labels_valid = final_labels[mask_valid]

    # Вычисляем метрику Calinski-Harabasz
    ch_score = calinski_harabasz_score(X_valid, labels_valid)
    # Вычисляем метрику Davies-Bouldin
    db_score = davies_bouldin_score(X_valid, labels_valid)

    print(f"Calinski-Harabasz Score: {ch_score:.2f}")
    print(f"Davies-Bouldin Score: {db_score:.2f}")

Calinski-Harabasz Score: 43867.78  
Davies-Bouldin Score: 0.34

### Визуализация итогового 2D графика кластеризации с Plotly

In [ ]:
pacmap_mapper = pacmap.PaCMAP(n_components=2, random_state=42, MN_ratio=30, FP_ratio=20)
embedding_2d = pacmap_mapper.fit_transform(embedding_intermediate_final)
# Центрируем 2D-проекцию вокруг [0, 0]
embedding_2d_centered = embedding_2d - embedding_2d.mean(axis=0)

In [ ]:
df_vis = pd.DataFrame({
    "Dim1": embedding_2d_centered[:, 0],
    "Dim2": embedding_2d_centered[:, 1],
    "Cluster": final_labels.astype(str)  # преобразуем метки в строку для категорий
})

fig_clusters = px.scatter(
    df_vis,
    x="Dim1",
    y="Dim2",
    color="Cluster",
    title="2D-проекция кластеризации (PaCMAP)",
    labels={"Dim1": "Dimension 1", "Dim2": "Dimension 2"}
)
fig_clusters.update_layout(legend_title_text="Кластер", height=800)
fig_clusters.show()

---

### Ray BOHB

In [16]:
from ray import tune
from ray.tune.schedulers import HyperBandForBOHB
from ray.tune.search.bohb import TuneBOHB
from ray.tune.search import ConcurrencyLimiter
from ray.tune import JupyterNotebookReporter, Checkpoint, CheckpointConfig
from ray.air import session
from ray.tune.callback import Callback

In [ ]:
def ray_objective(config, embeddings):
    reducer = UMAP(
        n_neighbors=int(config["n_neighbors"]),
        min_dist=config["min_dist"],
        n_components=int(config["n_components"]),
        metric="cosine"
    )
    embeddings_reduced = reducer.fit_transform(embeddings)

    clusterer = HDBSCAN(
        min_cluster_size=int(config["min_cluster_size"]),
        min_samples=int(config["min_samples"]),
        cluster_selection_epsilon=config["cluster_selection_epsilon"],
        cluster_selection_method="leaf",
        gen_min_span_tree=True,
        prediction_data=True
    )
    labels = clusterer.fit_predict(embeddings_reduced)
    score = calc_noiseless_silhouette(embeddings, embeddings_reduced, labels)
    tune.report({
        "silhouette_score": score,
        "umap_model": reducer,
        "hdbscan_model": clusterer
    })

##### Configuring tools for HPO

In [18]:
class SaveBestModels(Callback):
    """Class for saving the last pair of models with the highest value of metric"""
    def __init__(self):
        self.best_score = -float("inf")
        self.umap_path = "/home/denisalpino/ray_results/tune_bohb_umap_hdbscan/umap.pkl"
        self.hdbscan_path = "/home/denisalpino/ray_results/tune_bohb_umap_hdbscan/hdbscan.pkl"

    def on_trial_result(self, iteration, trial, result, **info):
        score = result["silhouette_score"]
        if score > self.best_score:
            self.best_score = score
            reducer = result["umap_model"]
            clusterer = result["hdbscan_model"]
            with open(self.umap_path, "wb") as f: pickle.dump(reducer, f)
            with open(self.hdbscan_path, "wb") as f: pickle.dump(clusterer, f)


In [ ]:
# Search space
search_space = {
    "n_neighbors": tune.choice(list(range(10, 60))),
    "n_components": tune.choice(list(range(10, 75))),
    "min_dist": tune.choice([i * 0.01 for i in range(0, 75)]),
    "min_cluster_size": tune.choice(list(range(10, 75))),
    "min_samples": tune.choice(list(range(5, 40))),
    "cluster_selection_epsilon": tune.choice([i * 0.01 for i in range(0, 30)]),
}

# BOHB Search + Scheduler setup
bohb_search = ConcurrencyLimiter(TuneBOHB(), max_concurrent=4)
bohb_sched = HyperBandForBOHB(time_attr="training_iteration", max_t=100, reduction_factor=4)

# Interactive reporter setup
reporter = JupyterNotebookReporter(
    overwrite=True,
    parameter_columns=[
        "n_neighbors", "n_components",
        "min_dist", "min_cluster_size",
        "min_samples", "cluster_selection_epsilon"
    ],
    metric_columns=["silhouette_score", "training_iteration"]
)

##### Launching HPO

In [ ]:
trainable = tune.with_parameters(ray_objective, embeddings=embeddings)

bohb_study = tune.run(
    trainable,
    config=search_space,
    metric="silhouette_score",
    mode="max",
    num_samples=50,
    search_alg=bohb_search,
    scheduler=bohb_sched,
    name="tune_bohb_umap_hdbscan",
    verbose=1,
    progress_reporter=reporter,
    resources_per_trial={"cpu": 1, "gpu": 1},
)

2025-04-24 01:30:23,053	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2025-04-24 01:31:36,679	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2025-04-24 01:32:02,852	INFO hyperband.py:543 -- Restoring from a previous point in time. Previous=1; Now=1
2025-04-24 01:32:04,246	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/denisalpino/ray_results/tune_bohb_umap_hdbscan' in 1.3815s.
2025-04-24 01:32:04,254	INFO tune.py:1041 -- Total run time: 101.20 seconds (99.79 seconds for the tuning loop).


### **Results**

##### **Printing results of optimization**

In [22]:
# Get the best hyperparameters
best_config = bohb_study.best_config
best_score = bohb_study.best_result["silhouette_score"]

# Load the best pair of models
with open("/home/denisalpino/ray_results/tune_bohb_umap_hdbscan/umap.pkl", "rb") as f:
    umap_best = pickle.load(f)
with open("/home/denisalpino/ray_results/tune_bohb_umap_hdbscan/hdbscan.pkl", "rb") as f:
    hdbscan_best = pickle.load(f)

# Calculate number of clusters and noise observtions
labels = hdbscan_best.labels_
n_clusters = len(np.unique(labels[labels != -1]))
n_noise = np.sum(labels == -1)

# Results
print(
    "TuneBOHB complited\n"
    "Clusterization rersults:\n"
    f"    clusters = {n_clusters},\n"
    f"    noise = {n_noise} observtions\n"
    f"The best Noiseless Silhouette score: {best_score:.4f}\n"
    "The best hyperparameters:"
)
for k, v in best_config.items():
    print(f"    {k} = {v},")

TuneBOHB complited
Clusterization rersults:
    clusters = 64,
    noise = 1639 observtions
The best Noiseless Silhouette score: 0.5164
The best hyperparameters:
    n_neighbors = 15,
    n_components = 68,
    min_dist = 0.58,
    min_cluster_size = 53,
    min_samples = 20,
    cluster_selection_epsilon = 0.24,


##### **HPO process vizualization**

In [ ]:
df_trials, fig = show_hpo(
    bohb_study,
    metric_name="silhouette_score",
    title="HPO combinations for UMAP & HDBSCAN (Bayesian Optimization with HyperBand)",
    lib="ray",
    top_n=50
)
fig.write_image(file="/home/denisalpino/dev/FinABYSS/notebooks/aspects/models/v4/hpo.jpg", format="jpg")

In [24]:
df_trials.head()

,n_neighbors,n_components,min_dist,min_cluster_size,min_samples,cluster_selection_epsilon,silhouette_score
1,15,68,0.58,53,20,0.24,0.516396
0,11,36,0.67,37,38,0.11,0.510807


In [ ]:
fig = show_feature_importance(df_trials)
fig.write_image(file="/home/denisalpino/dev/FinABYSS/notebooks/aspects/models/v4/hp_importance.jpg", format="jpg")